# Lab 3.2: Human-in-the-Loop (HITL) Basics

In this lab, we will implementation a **Real-World Customer Support Scenario**.

### The Scenario
We have an AI Agent that drafts responses to customer queries.
However, before sending the email, a **Human Manager** (you) must review and approve the draft.
If the draft is not good, the human can **edit** it before it is sent.

### Key Concepts
1. **Interrupts**: Pausing the graph before the email sending node.
2. **State Inspection**: Reviewing the AI's draft.
3. **State Update**: Manually correcting the draft.
4. **Resuming**: Allowing the "Send Email" node to execute with the corrected text.

In [ ]:
# 1. Install Dependencies
%pip install -qU langchain-groq langchain-community langgraph

In [ ]:
# 2. Setup API Keys
import getpass
import os

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

## 3. Define the Support Bot Graph

- **State**: Takes a `customer_query` and stores a `draft_response`.
- **Node 1 (`draft_reply`)**: Uses an LLM to generate a draft.
- **Node 2 (`send_email`)**: Simulates sending the final email.
- **Interrupt**: We will `interrupt_before` the `send_email` node.

### 3.1 Define State and Imports
We define the `SupportState` which holds the customer query, the draft response, and the final status.

In [ ]:
from typing import TypedDict, Optional
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_groq import ChatGroq

# --- 1. Define State ---
class SupportState(TypedDict):
    customer_query: str
    draft_response: Optional[str]
    final_status: Optional[str]

llm = ChatGroq(model_name="llama-3.3-70b-versatile")

### 3.2 Define Drafting Node
The `draft_reply` node uses the LLM to generate an initial email draft.

In [ ]:
def draft_reply(state: SupportState):
    print("--- (AI) Drafting Reply ---")
    query = state['customer_query']
    # Ask LLM to be polite and concise
    response = llm.invoke(f"Write a polite, concise support email response to this customer complaint: {query}")
    return {"draft_response": response.content}

### 3.3 Verification: Test Drafting Node
Let's test the drafting capability independently.

In [ ]:
# Verification
print("Testing draft_reply node...")
dummy_state = {"customer_query": "I received a broken product."}
draft_result = draft_reply(dummy_state)
print("Draft:", draft_result['draft_response'])

### 3.4 Define Send Email Node
This node simulates sending the email. In a real app, this would use an email API.

In [ ]:
def send_email(state: SupportState):
    print("--- (System) Sending Email ---")
    draft = state['draft_response']
    # Simulate sending
    return {"final_status": f"SENT: {draft}"}

### 3.5 Build and Compile Graph
We assemble the graph and add the **Interrupt** before the sending step.

In [ ]:
# --- 3. Build Graph ---
builder = StateGraph(SupportState)
builder.add_node("draft_reply", draft_reply)
builder.add_node("send_email", send_email)

builder.add_edge(START, "draft_reply")
builder.add_edge("draft_reply", "send_email")
builder.add_edge("send_email", END)

# --- 4. Compile with Interrupt ---
memory = MemorySaver()

# This is the key line: Pause BEFORE executing 'send_email'
graph = builder.compile(checkpointer=memory, interrupt_before=["send_email"])

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    pass

## 4. Run the Workflow (Phase 1: Drafting)
We start with a customer complaint about a broken widget.

In [ ]:
thread_config = {"configurable": {"thread_id": "ticket-123"}}

initial_input = {
    "customer_query": "My widget arrived broken! I am very angry. Fix this now!"
}

# Invoking the graph. It should STOP after 'draft_reply'.
print("Starting workflow...")
for event in graph.stream(initial_input, config=thread_config):
    print(event)

print("\nWorkflow paused?", graph.get_state(thread_config).next)

## 5. Human Review (Inspection)
The workflow is paused. As a manager, you need to check what the AI wrote.

In [ ]:
current_state = graph.get_state(thread_config)
ai_draft = current_state.values['draft_response']

print("--- Current AI Draft ---")
print(ai_draft)

## 6. Human Correction (Update State)
Imagine the AI was too generic. We want to add a specific promise, e.g., "We will ship a replacement immediately."
We use `update_state` to overwrite the `draft_response`.

In [ ]:
new_draft = ai_draft + "\n\nP.S. A replacement unit has been shipped via overnight delivery."

print("--- Updating Draft to ---")
print(new_draft)

graph.update_state(
    thread_config,
    {"draft_response": new_draft}
)

## 7. Resume Execution
Now that the state is corrected, we resume. The graph will pick up at `send_email` using our *new* draft.

In [ ]:
print("Resuming workflow...")
for event in graph.stream(None, config=thread_config):
    print(event)

# Check final result
final_state = graph.get_state(thread_config)
print("\nFinal Status:", final_state.values.get('final_status'))